In [ ]:
!wget http://vision.stanford.edu/Datasets/Stanford40_JPEGImages.zip

--2023-06-03 10:33:16--  http://vision.stanford.edu/Datasets/Stanford40_JPEGImages.zip
Resolving vision.stanford.edu (vision.stanford.edu)... 171.64.68.10
Connecting to vision.stanford.edu (vision.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 304771808 (291M) [application/zip]
Saving to: ‘Stanford40_JPEGImages.zip’

Stanford40_JPEGImag 100%[===================>] 290.65M  48.4MB/s    in 6.7s    

2023-06-03 10:33:23 (43.1 MB/s) - ‘Stanford40_JPEGImages.zip’ saved [304771808/304771808]



In [ ]:
!unzip Stanford40_JPEGImages.zip

Streaming output truncated to the last 5000 lines.
  inflating: JPEGImages/phoning_128.jpg  
  inflating: JPEGImages/phoning_129.jpg  
  inflating: JPEGImages/phoning_130.jpg  
  inflating: JPEGImages/phoning_131.jpg  
  inflating: JPEGImages/phoning_132.jpg  
  inflating: JPEGImages/phoning_133.jpg  
  inflating: JPEGImages/phoning_134.jpg  
  inflating: JPEGImages/phoning_135.jpg  
  inflating: JPEGImages/phoning_136.jpg  
  inflating: JPEGImages/phoning_137.jpg  
  inflating: JPEGImages/phoning_138.jpg  
  inflating: JPEGImages/phoning_139.jpg  
  inflating: JPEGImages/phoning_140.jpg  
  inflating: JPEGImages/phoning_141.jpg  
  inflating: JPEGImages/phoning_142.jpg  
  inflating: JPEGImages/phoning_143.jpg  
  inflating: JPEGImages/phoning_144.jpg  
  inflating: JPEGImages/phoning_145.jpg  
  inflating: JPEGImages/phoning_146.jpg  
  inflating: JPEGImages/phoning_147.jpg  
  inflating: JPEGImages/phoning_148.jpg  
  inflating: JPEGImages/phoning_149.jpg  
  inflating: JPEGImages/p

In [ ]:
import numpy as np
import os
import shutil
from glob import glob
import math
import tensorflow as tf
from keras import optimizers
from keras.models import Sequential
from keras.models import load_model
from keras.models import Model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import SpatialDropout2D
from keras.layers import Flatten, concatenate, GlobalAveragePooling2D
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import Lambda
from keras.layers import Concatenate
from keras.layers import LeakyReLU
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Permute
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import CSVLogger
from keras import layers, models
from keras import backend as K
import matplotlib.pyplot as plt

In [ ]:
for file in os.listdir('JPEGImages'):
    p=file.rfind('_')
    img_class=file[:p]

    if not os.path.exists(f'unsplit_data/{img_class}'):
        os.makedirs(f'unsplit_data/{img_class}')

    shutil.copy(f'JPEGImages/{file}',f'unsplit_data/{img_class}/{file}')

In [ ]:
os.listdir("unsplit_data")

['pouring_liquid',
 'jumping',
 'throwing_frisby',
 'fixing_a_bike',
 'fixing_a_car',
 'fishing',
 'walking_the_dog',
 'reading',
 'applauding',
 'climbing',
 'playing_violin',
 'blowing_bubbles',
 'pushing_a_cart',
 'writing_on_a_board',
 'gardening',
 'waving_hands',
 'brushing_teeth',
 'playing_guitar',
 'watching_TV',
 'cooking',
 'looking_through_a_telescope',
 'texting_message',
 'holding_an_umbrella',
 'phoning',
 'drinking',
 'washing_dishes',
 'cutting_trees',
 'cleaning_the_floor',
 'smoking',
 'feeding_a_horse',
 'riding_a_bike',
 'writing_on_a_book',
 'cutting_vegetables',
 'rowing_a_boat',
 'using_a_computer',
 'looking_through_a_microscope',
 'running',
 'shooting_an_arrow',
 'riding_a_horse',
 'taking_photos']

In [ ]:
!pip install split-folders

import splitfolders

splitfolders.ratio("unsplit_data", output="data", seed=1337, ratio=(0.8, 0.2))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Copying files: 9532 files [00:01, 6453.54 files/s]


In [ ]:
# dimensions of our images.
img_width, img_height = 256, 256

train_data_dir = "data/train"
validation_data_dir = "data/val"

nb_train_samples = 7611
nb_validation_samples = 1921
epochs = 100
batch_size = 16


In [ ]:
# train model on a training dataset
def train_model(model, train_generator, validation_generator):

    # compile model
    model.compile(
        loss="categorical_crossentropy", optimizer="sgd", metrics=["accuracy"]
    )

    # fit model
    history = model.fit(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size,
    )

    return history


In [ ]:
from matplotlib import pyplot as plt

# summarize the performance of the fit model
def summarize_model(model, history, test_generator):
    # evaluate the model
    _, test_acc = model.evaluate(test_generator, verbose=0)
    print("Test: %.3f" % (test_acc))

    # plot loss during training
    plt.subplot(211)
    plt.title("Loss")
    plt.plot(history.history["loss"], label="train")
    plt.plot(history.history["val_loss"], label="test")
    plt.legend()

    # plot accuracy during training
    plt.subplot(212)
    plt.title("Accuracy")
    plt.plot(history.history["accuracy"], label="train")
    plt.plot(history.history["val_accuracy"], label="test")
    plt.legend()
    plt.show()


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    fill_mode="nearest",
    rescale=1.0 / 255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.5, 1.5],
    shear_range=0.2,
    rotation_range=30,
    zoom_range=0.2,
    horizontal_flip=True,
)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="categorical",
)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="categorical",
)


Found 7611 images belonging to 40 classes.
Found 1921 images belonging to 40 classes.


#INCEPTIONETV3 (Upayan)


In [ ]:
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten

# example of loading the inception v3 model
from keras.applications.inception_v3 import InceptionV3

# define model
model = InceptionV3(include_top=False, input_shape=(256, 256, 3))

flat1 = Flatten()(model.output)
class1 = Dense(1024, activation="relu")(flat1)
output = Dense(40, activation="softmax")(class1)

model = Model(inputs=model.input, outputs=output)

#display model architecture
print(model.summary())

87910968/87910968 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 127, 127, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 127, 127, 32  96         ['conv2d[0][0]']                 
 alization)                  

In [ ]:
# fit model on train dataset
history = train_model(model, train_generator, validation_generator)

Epoch 1/100
 96/475 [=====>........................] - ETA: 2:32 - loss: 3.6360 - accuracy: 0.1156

In [ ]:
# evaluate model behavior
summarize_model(model, history, validation_generator)

#DENSENET 201(Farhan)

##Attention Mechanism

In [ ]:
data_format = K.image_data_format()
K.set_image_data_format(data_format)
np.random.seed(7)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
num_of_classes = 40

In [ ]:
from keras.applications import DenseNet201

model = DenseNet201(
    include_top=False,
    input_shape=(224,224,3),
)

In [ ]:
model.summary()

In [ ]:
conv_base = Model(inputs=model.inputs, outputs=model.get_layer('relu').output)

In [ ]:
def spatial_attention(input_feature):
    kernel_size = 7
    if K.image_data_format() == "channels_first":
        channel = input_feature.shape[1]
        cbam_feature = Permute((2, 3, 1))(input_feature)
    else:
        channel = input_feature.shape[-1]
        cbam_feature = input_feature

    avg_pool = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(cbam_feature)
    assert avg_pool.shape[-1] == 1
    max_pool = Lambda(lambda x: K.max(x, axis=3, keepdims=True))(cbam_feature)
    assert max_pool.shape[-1] == 1
    concat = Concatenate(axis=3)([avg_pool, max_pool])
    assert concat.shape[-1] == 2
    cbam_feature = Conv2D(filters=1,
                          kernel_size=kernel_size,
                          strides=1,
                          padding='same',
                          activation='sigmoid',
                          kernel_initializer='he_normal',
                          use_bias=False)(concat)
    assert cbam_feature.shape[-1] == 1

    if K.image_data_format() == "channels_first":
        cbam_feature = Permute((3, 1, 2))(cbam_feature)
    x1 = concatenate([input_feature, cbam_feature])

    return x1
    # return cbam_feature

In [ ]:
def Trunc_DenseNet201(conv_base):
    conv_base.trainable = True
    layer_4th = conv_base.get_layer('conv5_block32_concat').output
    attention_module = spatial_attention(layer_4th)
    print(attention_module.shape)
    flatten = Flatten()(attention_module)
    # flatten=GlobalAveragePooling2D()(attention_module)
    dense = Dense(2048, activation='relu')(flatten)
    # dropout = Dropout(0.5)(dense)
    output = BatchNormalization()(dense)
    dense = Dense(1024, activation='relu')(output)
    # dropout = Dropout(0.3)(dense)
    output = BatchNormalization()(dense)
    dense = Dense(512, activation='relu')(output)
    dropout = Dropout(0.3)(dense)
    output = BatchNormalization()(dropout)
    dense = Dense(256, activation='relu')(output)
    dropout = Dropout(0.5)(dense)
    output = BatchNormalization()(dropout)
    dense = Dense(24, activation='relu')(output)
    pred = (Dense(num_of_classes, activation='softmax'))(dense)
    model = Model(inputs=conv_base.inputs, outputs=pred)
    return model

In [ ]:
model = Trunc_DenseNet201(conv_base)

In [ ]:
model.summary()

In [ ]:
# train model on a training dataset
def train_model(model, train_generator, validation_generator):

    # compile model
        sgd = optimizers.SGD(learning_rate = 0.01, momentum = 0.9, clipnorm = 1.0)
        adam = optimizers.Adam(learning_rate=0.001)
        model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    # fit model
        history = model.fit(
          train_generator,
          steps_per_epoch=nb_train_samples // batch_size,
          epochs=epochs,
          validation_data=validation_generator,
          validation_steps=nb_validation_samples // batch_size,
    )

        return history


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
                                   #samplewise_center=True,
                                   #samplewise_std_normalization=True
                                   fill_mode="nearest",
                                   rescale=1.0 / 255,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   brightness_range=[0.5, 1.5],
                                   shear_range=0.2,
                                   rotation_range=30,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   )
test_datagen = ImageDataGenerator(
                                   rescale=1.0 / 255,
                                   #samplewise_center=True,
                                   #samplewise_std_normalization=True
                                 )

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224,224),
    batch_size=batch_size,
    class_mode="categorical",
)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(224,224),
    batch_size=batch_size,
    class_mode="categorical",
)


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
checkpoint = ModelCheckpoint(monitor='val_accuracy', verbose=1, filepath='/content/drive/MyDrive/densenet201.model.h5', save_best_only=True, mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.7, min_delta = 0.0005,
                              patience=20, min_lr=0.001, verbose = 1)
callbacks_list = [checkpoint,reduce_lr]

In [ ]:
# fit model on train dataset
history = train_model(model, train_generator, validation_generator)

In [ ]:
# evaluate model behavior
summarize_model(model, history, validation_generator)

## Loading Saved Model and Resume Training


In [ ]:
model.save("/content/drive/MyDrive/densenet201.model.h5")

In [ ]:
history = train_model(saved_model, train_generator, validation_generator)

In [ ]:
# evaluate model behavior
summarize_model(saved_model, history, validation_generator)

#*Attention (InceptionV3)*

In [ ]:
data_format = K.image_data_format()
K.set_image_data_format(data_format)
np.random.seed(7)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten

# example of loading the inception v3 model
from keras.applications.inception_v3 import InceptionV3

# define model
model = InceptionV3(include_top=False, input_shape=(256, 256, 3))

#display model architecture
print(model.summary())

87910968/87910968 [==============================] - 0s 0us/step
Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 127, 127, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 127, 127, 32  96         ['conv2d[0][0]']                 
 alization)           

In [ ]:
num_of_classes = 40

In [ ]:
conv_base = Model(inputs=model.inputs, outputs=model.get_layer('mixed10').output)

In [ ]:
def spatial_attention(input_feature):
    kernel_size = 7
    if K.image_data_format() == "channels_first":
        channel = input_feature.shape[1]
        cbam_feature = Permute((2, 3, 1))(input_feature)
    else:
        channel = input_feature.shape[-1]
        cbam_feature = input_feature

    avg_pool = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(cbam_feature)
    assert avg_pool.shape[-1] == 1
    max_pool = Lambda(lambda x: K.max(x, axis=3, keepdims=True))(cbam_feature)
    assert max_pool.shape[-1] == 1
    concat = Concatenate(axis=3)([avg_pool, max_pool])
    assert concat.shape[-1] == 2
    cbam_feature = Conv2D(filters=1,
                          kernel_size=kernel_size,
                          strides=1,
                          padding='same',
                          activation='sigmoid',
                          kernel_initializer='he_normal',
                          use_bias=False)(concat)
    assert cbam_feature.shape[-1] == 1

    if K.image_data_format() == "channels_first":
        cbam_feature = Permute((3, 1, 2))(cbam_feature)
    x1 = concatenate([input_feature, cbam_feature])

    return x1
    # return cbam_feature

In [ ]:
def Trunc_InceptionV3(conv_base):
    conv_base.trainable = True
    layer_4th = conv_base.get_layer('mixed10').output
    attention_module = spatial_attention(layer_4th)
    print(attention_module.shape)
    flatten = Flatten()(attention_module)
    # flatten=GlobalAveragePooling2D()(attention_module)
    dense = Dense(2048, activation='relu')(flatten)
    # dropout = Dropout(0.5)(dense)
    output = BatchNormalization()(dense)
    dense = Dense(1024, activation='relu')(output)
    # dropout = Dropout(0.3)(dense)
    output = BatchNormalization()(dense)
    dense = Dense(512, activation='relu')(output)
    dropout = Dropout(0.3)(dense)
    output = BatchNormalization()(dropout)
    dense = Dense(256, activation='relu')(output)
    dropout = Dropout(0.5)(dense)
    output = BatchNormalization()(dropout)
    dense = Dense(24, activation='relu')(output)
    pred = (Dense(num_of_classes, activation='softmax'))(dense)
    model = Model(inputs=conv_base.inputs, outputs=pred)
    return model

In [ ]:
model = Trunc_InceptionV3(conv_base)

(None, 6, 6, 2049)


In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 127, 127, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 127, 127, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                           

In [ ]:
# train model on a training dataset
def train_model(model, train_generator, validation_generator):

    # compile model
        sgd = optimizers.SGD(learning_rate = 0.01, momentum = 0.9, clipnorm = 1.0)
        adam = optimizers.Adam(learning_rate=0.001)
        model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    # fit model
        history = model.fit(
          train_generator,
          steps_per_epoch=nb_train_samples // batch_size,
          epochs=epochs,
          validation_data=validation_generator,
          validation_steps=nb_validation_samples // batch_size,
    )

        return history


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
                                   #samplewise_center=True,
                                   #samplewise_std_normalization=True
                                   fill_mode="nearest",
                                   rescale=1.0 / 255,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   brightness_range=[0.5, 1.5],
                                   shear_range=0.2,
                                   rotation_range=30,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   )
test_datagen = ImageDataGenerator(
                                   rescale=1.0 / 255,
                                   #samplewise_center=True,
                                   #samplewise_std_normalization=True
                                 )

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(256,256),
    batch_size=batch_size,
    class_mode="categorical",
)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(256,256),
    batch_size=batch_size,
    class_mode="categorical",
)


Found 7611 images belonging to 40 classes.
Found 1921 images belonging to 40 classes.


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
checkpoint = ModelCheckpoint(monitor='val_accuracy', verbose=1, filepath='/content/drive/MyDrive/inceptionV3.model.h5', save_best_only=True, mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.7, min_delta = 0.0005,
                              patience=20, min_lr=0.001, verbose = 1)
callbacks_list = [checkpoint,reduce_lr]

In [ ]:
# fit model on train dataset
history = train_model(model, train_generator, validation_generator)

Epoch 1/100
475/475 [==============================] - 217s 402ms/step - loss: 3.7731 - accuracy: 0.0248 - val_loss: 3.6827 - val_accuracy: 0.0344
Epoch 2/100
475/475 [==============================] - 185s 388ms/step - loss: 3.7023 - accuracy: 0.0294 - val_loss: 5.3714 - val_accuracy: 0.0224
Epoch 3/100
475/475 [==============================] - 188s 397ms/step - loss: 3.6882 - accuracy: 0.0308 - val_loss: 3.6959 - val_accuracy: 0.0464
Epoch 4/100
475/475 [==============================] - 187s 393ms/step - loss: 3.6723 - accuracy: 0.0324 - val_loss: 3.8130 - val_accuracy: 0.0339
Epoch 5/100
475/475 [==============================] - 188s 395ms/step - loss: 3.6644 - accuracy: 0.0323 - val_loss: 4.6344 - val_accuracy: 0.0427
Epoch 6/100
475/475 [==============================] - 187s 394ms/step - loss: 3.6529 - accuracy: 0.0409 - val_loss: 4.0179 - val_accuracy: 0.0365
Epoch 7/100
475/475 [==============================] - 187s 394ms/step - loss: 3.6403 - accuracy: 0.0406 - val_loss: 3

In [ ]:
# evaluate model behavior
summarize_model(model, history, validation_generator)